In [ ]:
!pip3 install matplotlib --break-system-packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io.wavfile as wav
import scipy.signal

from IPython.display import Audio


In [ ]:
def wavread(filename):
  """Read in audio data from a wav file.  Return d, sr."""
  # Read in wav file.
  file_handle = open(filename, 'rb')
  samplerate, wave_data = wav.read(file_handle)
  # Normalize short ints to floats in range [-1..1).
  data = np.asarray(wave_data, dtype=np.float32) / 32768.0
  return data, samplerate

def wavwrite(data, samplerate, filename):
  """Write a waveform to a WAV file."""
  wav.write(filename, samplerate, (32768.0 * data).astype(np.int16))

def play(waveform, rate):
    int

def read_and_trim(filename, duration=2.0, rel_threshold=0.005, abs_threshold=0, sr=44100, channel=None):
    d, file_sr = wavread(filename)
    print("d.shape=", d.shape)
    if len(d.shape) > 1:  # Stereo file
        if channel is None:
            d = np.mean(d, axis=-1)
        else:
            d = d[:, channel]
    assert file_sr == sr
    d = d - np.mean(d)
    t = np.arange(len(d)) / sr
    plt.figure(figsize=(12, 4))
    plt.subplot(121)
    plt.plot(t[:50000], d[:50000])
    threshold = np.maximum(np.max(np.abs(d)) * rel_threshold, abs_threshold)
    plt.ylim(threshold * np.array([-1, 1]))
    plt.subplot(122)
    plt.plot(t, d)
    # Tight window at start?
    drop_initial_samps = np.maximum(0, -50 + np.min(np.flatnonzero(np.abs(d) > threshold)))
    print("drop initial", drop_initial_samps / sr)
    d = d[int(round(drop_initial_samps)) + np.arange(int(round(duration * sr)))]
    return d, sr

filename = 'Piano.ff.D4.wav'
waveform, sr = read_and_trim(filename, channel=0, rel_threshold=0.005, abs_threshold=0.0002)
#Audio(data=waveform.T, rate=sr)

In [ ]:
plt.plot(np.arange(len(waveform)) / sr, waveform)

In [ ]:
def freq_from_autoco(d, sr=44100):
  # Estimate fundamental by autocorrelation.
  acr = np.correlate(d, d, 'full')[len(d) - 1: len(d) + 10000]
  # Find Nth peak, divide by N
  maxima = scipy.signal.argrelextrema(acr, np.greater)[0]
  npeaks = 100
  freqs = sr * np.arange(1, npeaks + 1) / maxima[:npeaks]
  #plt.plot(freqs)
  freq = np.mean(freqs[-40:])
  return freq

freq = freq_from_autoco(waveform)
print(freq)
# Nominal: D5= 587.3295

In [ ]:
def lin_to_db(l):
    return 20 * np.log10(l)

def db_to_lin(d):
    return np.pow(10.0, d / 20.0)

In [ ]:
def heterodyne_extract(waveform, fundamental_freq, harmonic_number=1.0, sr=44100, smooth_cycles=2.0):
    """Return a full-sample-rate amplitude and frequency envelope near the specified frequency."""
    t = np.arange(len(waveform)) / sr
    h_freq = harmonic_number * fundamental_freq
    complex_exp = np.exp(-1j * 2 * np.pi * h_freq * t)
    f_period = sr / (h_freq / harmonic_number)
    #print("f_period=", f_period)
    smooth_len = int(round(smooth_cycles * f_period * harmonic_number))
    #print("sm_len=", smooth_len)
    smooth_win = np.hanning(smooth_len)
    smooth_win = smooth_win / np.sum(smooth_win)
    smoothed_heterodyne_waveform = np.convolve(smooth_win, waveform * complex_exp, 'same')
    heterodyne_magnitude = np.abs(smoothed_heterodyne_waveform)
    smoothed_dphi_dt = np.diff(np.convolve(smooth_win, np.hstack([[0], np.unwrap(np.angle(smoothed_heterodyne_waveform))]), 'same'))
    inst_freq = h_freq + smoothed_dphi_dt / (2 * np.pi) * sr
    dphi_dt_valid = (np.abs(smoothed_dphi_dt) < 0.5)
    dphi_dt_weighting = heterodyne_magnitude * dphi_dt_valid
    weighted_mean_dphi_dt = np.sum(dphi_dt_weighting * smoothed_dphi_dt) / np.sum(dphi_dt_weighting)   # radians / sample
    avg_freq = h_freq + weighted_mean_dphi_dt / (2 * np.pi) * sr   # cycles / second
    reconstructed_waveform = 2 * np.real(smoothed_heterodyne_waveform * np.conj(complex_exp))
    return lin_to_db(heterodyne_magnitude), inst_freq, avg_freq, reconstructed_waveform

In [ ]:
print(np.reshape(np.tile(np.array([[1,2,3,4],[5,6,7,8]])[:,:, np.newaxis], [1, 1, 2]), shape=(2, 8)))

def replicate_last(x, num_reps):
    shape_in = x.shape
    shape_out = np.array(shape_in)
    shape_out[-1] *= num_reps
    return np.reshape(np.tile(x[..., np.newaxis], list(np.ones_like(shape_in)) + [num_reps]), shape=shape_out)

print(replicate_last(np.array([[1,2,3,4],[5,6,7,8]]), 5))

In [ ]:
def slinterp(x, interp):
    x_shape = list(x.shape)
    x_out_shape = list(x.shape)
    x_out_shape[-1] *= interp
    x_in_flat = np.reshape(x, (np.prod([1] + x_shape[:-1]), x_shape[-1]))
    n_rows, n_cols = x_in_flat.shape
    x_out_flat = np.zeros((n_rows, interp * n_cols))
    interpolation_window = np.hstack([np.linspace(0, 1, 1 + interp)[1:], np.linspace(1, 0, 1 + interp)[1:]])
    for i in range(x_in_flat.shape[0]):
        x_atrous = np.reshape(np.hstack([x_in_flat[[i], :].T, np.zeros((n_cols, interp - 1))]), (n_cols * interp,))
        x_out_flat[i] = np.convolve(x_atrous, interpolation_window, 'same')
    return np.reshape(x_out_flat, x_out_shape)

plt.plot(slinterp(np.array([[0, 1, 0, 2, 2]]), 5).T, '.')
plt.grid()

In [ ]:
def synth_partial(mag, freq, frame_rate, sr):
    samps_per_frame = int(round(sr / frame_rate))
    num_partials, num_frames = mag.shape
    output_samples = num_frames * samps_per_frame
    t = np.arange(output_samples) / sr
    dt = 1 / sr
    #phase = np.cumsum(np.hstack([np.zeros((num_partials, 1)), slinterp(2 * np.pi * freq * dt, samps_per_frame)]))
    phase = np.cumsum(slinterp(2 * np.pi * freq * dt, samps_per_frame))
    carrier = np.cos(phase)
    return slinterp(mag, samps_per_frame) * carrier

x = synth_partial(np.array([[1, 2, 3, 2]]), np.array([[40, 80, 120, 20]]), 10, 1000)
plt.figure(figsize=(18, 4))
plt.plot(x.T)
plt.grid()
print(x.shape)

In [ ]:
# Heterodyne
# h_num  delta_f
# 1      -0.63
# 2      -6.5
# 3       4.1
# 4      18.7
# 5      35.3
# 6      61.4
# 7     104     4229
# 8.2    31.4   4863
# 9.3    97.1   5518
# 10.2          6003
# 10.8          6222
# 11.2          6593
# 11.7          6890
# 13.3          7830
# 14.15         8343
# 15.45         9097
# 16.7          9845

f_freq = 589.232
harmonic_num = 1
sm_cyc = 2.0

mag, freq, avg_freq, recons_waveform = heterodyne_extract(waveform, f_freq, harmonic_num, sr, sm_cyc)

t = np.arange(len(waveform)) / sr
ii = np.arange(2000) # 
ii = np.arange(len(mag)-1000,len(mag))

print(mag.shape)
print(freq.shape)

plt.subplot(211)
#plt.plot(t[ii], db_to_lin(mag[ii]), '.')
plt.plot(t[ii], mag[ii], '.')
plt.grid()
#plt.ylim([-60, -20])

plt.subplot(212)
plt.plot(t[ii], freq[ii], '.')
plt.ylim([0.9 * avg_freq, 1.1 * avg_freq])
plt.grid()

print("pre_freq=%.2f" % (f_freq * harmonic_num), "post_freq=%.2f" % avg_freq, "diff=%.2f" % (avg_freq - f_freq * harmonic_num))

#Audio(data=synth_partial(np.array([db_to_lin(mag)]), np.array([freq]), sr, sr), rate=sr)
#Audio(data=synth_partial(np.array([db_to_lin(mag)]), avg_freq * np.ones((1, len(freq))), sr, sr), rate=sr)

In [ ]:
from scipy import fft

def localmax(x, npts):
    """Return indices of x where value is largest among nearest npts."""
    maxima = []
    for i in np.arange(1, len(x)):
        locality = x[np.maximum(0, i - npts // 2) : np.minimum(len(x), i + npts // 2)]
        if x[i] == np.max(locality) and x[i] > x[i - 1]:
            maxima.append(i)
    return np.array(maxima)


def peak_freqs(x, n_fft=65536, f_max=10000.0, depth_db=20.0, local_smooth_points=2047, local_max_points=100, sr=44100):
    """Return frequencies of local maxima from a long FFT."""
    long_fft = fft.fft(x[:n_fft])
    long_fft = long_fft[:(n_fft // 2)]
    freqs = np.arange(n_fft // 2) * sr / n_fft
    n_bins = np.sum(freqs < f_max)
    X_spec = 20 * np.log10(np.abs(long_fft[:n_bins]))
    # Local average
    smoo_X_spec = np.convolve(np.ones(local_smooth_points) / local_smooth_points, X_spec, 'same')
    unsmoo_X_spec = X_spec - smoo_X_spec

    plt.figure(figsize=(20, 6))
    plt.plot(freqs[:n_bins], unsmoo_X_spec)

    #peaks = scipy.signal.argrelextrema(np.maximum(np.max(unsmoo_X_spec) - 30, unsmoo_X_spec), np.greater)[0]
    peaks = localmax(np.maximum(np.max(unsmoo_X_spec) - depth_db, unsmoo_X_spec), local_max_points)

    plt.plot(freqs[peaks], unsmoo_X_spec[peaks], 'or')
    
    return peaks * sr / n_fft

pk_frqs = peak_freqs(waveform, depth_db=30)

print(pk_frqs)
print(pk_frqs / pk_frqs[0])


In [ ]:
def extract_harmonics(d, num_harms=8):

    recons_d = np.zeros_like(d)
    recons_d_flat_f = np.zeros_like(d)

    sm_cyc = 2.0

    d_remaining = np.array(d)

    mags = []
    freqs = []
    avg_freqs = []

    #h_num_list = [1, 2, 3, 4, 5, 6, 7.2, 8.2]  #, 9.3, 10.2, 10.55, 10.8, 11.2, 11.7, 13.3, 14.15, 15.45, 16.7]
    #h_num_list = 1 + np.arange(num_harms)
    f_peaks = peak_freqs(d, depth_db=30)[:num_harms]
    print(f_peaks)
    f_0 = f_peaks[0]
    h_num_list = f_peaks / f_0
    
    for h_num in h_num_list:
        mag, freq, avg_freq, recons_waveform = heterodyne_extract(d_remaining, f_0, h_num, sr, sm_cyc)
        # Repeat with estimated frequency
        print(h_num, avg_freq / h_num, f_freq * h_num, avg_freq)
        if h_num > 8:
            mag, freq, avg_freq, recons_waveform = heterodyne_extract(d, avg_freq / h_num, h_num, sr, sm_cyc)
        recons_d += recons_waveform
        d_remaining -= recons_waveform
        recons_d_flat_f += synth_partial(np.array([db_to_lin(mag)]), avg_freq * np.ones((1, len(freq))), sr, sr)[0]
        mags.append(mag)
        freqs.append(freq)
        avg_freqs.append(avg_freq)
    
    mags = np.array(mags)
    freqs = np.array(freqs)
    print(mags.shape)

    return mags, recons_d, freqs, avg_freqs

d = waveform

mags, recons_d, freqs, avg_freqs = extract_harmonics(d, num_harms=16)
resid = d - recons_d

t = np.arange(len(d)) / sr
ii = np.arange(14000, 14200)
plt.plot(t[ii], d[ii], '.', t[ii], recons_d[ii], '.', t[ii], resid[ii])
#Audio(data=d.T, rate=sr)
#Audio(data=recons_d.T, rate=sr)
Audio(data=(resid).T, rate=sr)
#Audio(data=recons_d_flat_f.T, rate=sr)

In [ ]:
wavwrite(resid, sr, 'resid.wav')
wavwrite(recons_d, sr, 'recons.wav')

In [ ]:
# Fit inharmonicity curve to effective fundamentals
xx = 1 + np.arange(len(avg_freqs))
h_freq = avg_freqs
plt.plot(xx, h_freq / xx, '.')
plt.plot(xx, h_freq[0] * np.sqrt(1 + .001 * xx*xx))  # for D5
plt.plot(xx, h_freq[0] * np.sqrt(1 + .0004 * xx*xx)) # for D4
#plt.ylim(np.array([.98, 1.2]) * h_freq[0])

In [ ]:
from scipy import fft

def spectrogram(x, fft_len=1024, win_len=None, hop_len=None, window_fn=np.hanning, sr=1, f_max=0.5, title=None):
    if win_len is None:
        win_len = fft_len
    if hop_len is None:
        hop_len = win_len // 2
    # Window.
    prepad_len = (fft_len - win_len) // 2
    win = np.hstack([np.zeros(prepad_len), window_fn(win_len), np.zeros(fft_len - win_len - prepad_len)])
    # Frame.
    frame_indices = np.arange(0, len(x) - win_len, hop_len)[:, np.newaxis] + np.arange(win_len)[np.newaxis, :]
    x_chunks_windowed = x[frame_indices] * win[np.newaxis, :]
    # Transform.
    stft_mag_db = 20 * np.log10(np.abs(fft.fft(x_chunks_windowed, n=fft_len)[:, :(fft_len // 2 + 1)]))
    num_frames, num_bins = stft_mag_db.shape
    t_base = np.arange(num_frames) * hop_len / sr
    f_base = np.arange(num_bins) * sr / 2
    plt.imshow(stft_mag_db.T, aspect='auto', origin='lower')
    plt.clim(np.max(stft_mag_db) + [-80, 0])
    plt.ylim([0, f_max / sr * fft_len])
    x_times = np.arange(0, len(x) / sr, 0.25)
    y_freqs = np.arange(0, 10000, 1000)
    plt.xticks(x_times * sr / hop_len, x_times)
    plt.yticks(y_freqs / sr * fft_len, y_freqs)
    plt.colorbar()
    if title:
        plt.title(title)

plt.figure(figsize=(18, 12))
plt.subplot(311)
spectrogram(d, sr=sr, f_max=10000, title='original')
plt.subplot(312)
spectrogram(recons_d, sr=sr, f_max=10000, title='partials')
plt.subplot(313)
spectrogram(resid, sr=sr, f_max=10000, title='residual')
#spectrogram(recons_d_flat_f, sr=sr, f_max=10000, title='flat_f partials')


In [ ]:
# Piecewise-linear fits to amplitude envelopes.
# from piano-partials.ipynb

def lin_fit(x):
    """Least-squares linear fit to points."""
    # xhat = a + b.index
    # err = x[i] - (a + b.i)
    # err^2 = x[i]^2 - 2 x[i] (a + b.i) + (a^2 + 2 a b i +b^2 i^2)
    # Define axes around midpoint.
    lenx = len(x)
    if not lenx:
        return []
    index = np.arange(lenx) - (lenx - 1) / 2
    # Make x zero mean
    a = np.mean(x)
    x_z = x - a
    b = 0
    if lenx > 1:
        # Linear fit is normalized inner product
        b = np.sum(x_z * index) / np.sum(index * index)
    return a + b * index

def fit_2_segs(x):
    """Fit x with 2 linear segments, searching all divisions."""
    # Try every poss division of remainder.
    besterr = np.sum(np.abs(x))
    besterrpt = 0
    bestxhat = []
    bestseg1 = []
    bestseg2 = []
    for divpt in np.arange(0, len(x)):
        seg1 = lin_fit(x[:divpt + 1])
        seg2 = lin_fit(x[divpt:])
        xhat = np.hstack([seg1[:-1], seg2])
        abserr = np.sum(np.abs(xhat - x))
        if abserr < besterr:
            besterr = abserr
            besterrpt = divpt
            bestseg1 = seg1
            bestseg2 = seg2
            bestxhat = xhat
    # Parameterization is [value, interval, value interval, value...]
    parameters = [bestseg1[0], len(bestseg1), 0.5 * (bestseg1[-1] + bestseg2[0]), len(bestseg2), bestseg2[-1]]
    #print(bestseg1, bestseg2, parameters)
    #print("parameters=", parameters)
    return bestxhat, parameters

def expand_params(params):
    """Convert [val, npts, val, npts, val] sequence to points."""
    val0 = params[0]
    outpts = [[val0]]
    npts_val_pairs = np.vstack([params[1::2], params[2::2]])
    for npts, val in npts_val_pairs.transpose():
        outpts.append(np.linspace(val0, val, int(npts))[1:])
        val0 = val
    return np.concatenate(outpts)

def linseg_fit(x):
    """Approximate sequence x as a set of line segments."""
    # Start with onset slope
    maxpt = np.argmax(x)
    #print("maxpt=", maxpt)
    initial, i_params = fit_2_segs(x[:maxpt + 1])
    tail, t_params = fit_2_segs(x[maxpt:])
    params = i_params[:-1] + [0.5 * (i_params[-1] + t_params[0])] + t_params[1:]
    #print(initial.shape, tail.shape)
    #print(params)
    return np.hstack([initial, tail[1:]]), expand_params(params), params


#i = np.arange(len(mag))
#plt.plot(i[:2000], mag[:2000], i[:2000:64], mag[:2000:64])

hop_len = 64

trim_ends_sec = 0.014
trim_ends_samps = int(round(trim_ends_sec * sr))

hnum = 2
mag_trim = mags[hnum][trim_ends_samps:-trim_ends_samps]

xhat, xhat2, params = linseg_fit(mag_trim[::hop_len])

t = np.arange(len(mag_trim)) / sr
plt.plot(t, mag_trim, t[::hop_len], xhat, t[::hop_len], xhat2)
plt.xlim([0, 0.1])
print(params)

In [ ]:
# New approach to linseg fitting - take all extrema, then filter down.
filt_len = 31  # for D5
#filt_len = 127  # for D4
#filt_len = 1023
filt_win = np.hanning(filt_len)/np.sum(np.hanning(filt_len))

def lin_hull(x, ends=None):
    if ends is None:
        ends = (x[0], x[-1])
    npts = len(x)
    return np.linspace(ends[0], ends[1], npts)

def max_prominence(x, ends=None, polarity=1):
    prominence = polarity * (x - lin_hull(x, ends))
    argmax_prominence = np.argmax(prominence)
    if np.max(prominence) > 0:
        return argmax_prominence
    return None

def err_from_line(x, ends=None, abs_fn=np.abs):
    """Return area between x and line seg defined by ends."""
    return np.sum(abs_fn(x - lin_hull(x, ends)))

def convex_hull(x, points=None, polarity=1):
    if points is None:
        points = [0, len(x) - 1]
    new_points = [0]
    for start in range(len(points) - 1):
        new_point = max_prominence(x[points[start]:points[start + 1] + 1], polarity)
        if new_point:
            new_points.append(points[start] + int(new_point))
        new_points.append(points[start + 1])
    return new_points

def hull(x, npoints):
    points = [0, len(x) - 1]
    while len(points) < npoints:
        max_err = 0
        for i in range(len(points) - 1):
            start = points[i]
            end = points[i + 1]
            err = err_from_line(x[start : end]) / np.sqrt((end - start))
            #print(points, start, end, err, max_err)
            if abs(err) > max_err:
                max_err = abs(err)
                worst_seg_ix = i
        worst_start = points[worst_seg_ix]
        worst_end = points[worst_seg_ix + 1]
        polarity = 1 if err_from_line(x[worst_start : worst_end], abs_fn=np.asarray) > 0 else -1
        points.insert(worst_seg_ix + 1, int(worst_start + max_prominence(x[worst_start : worst_end], polarity=polarity)))
    return np.array(points)

#h = convex_hull(mag_trim)
#print(h)
#h = convex_hull(mag_trim, h, -1)
#print(h)
#h = convex_hull(mag_trim, h)
#print(h)
#h = convex_hull(mag_trim, h, -1)
#print(h)
#h = convex_hull(mag_trim, h)
#print(h)

trim_start_samps = 128
trim_end_samps = 1024

hnum = 11
mag_filt = lin_to_db(np.convolve(filt_win, db_to_lin(mags[hnum]), 'same'))
mag_filt_trim = mag_filt[trim_start_samps:-trim_end_samps]
max_times = scipy.signal.argrelmax(mag_filt_trim)[0]
max_vals = mag_filt_trim[max_times]
t = np.arange(len(mag_filt_trim)) / sr
mmag = np.maximum(-130.0, mag_filt_trim)
h = hull(mmag, 12)
plt.figure(figsize=(16, 5))
plt.subplot(121)
plt.plot(t*sr, mmag, t[max_times]*sr, max_vals, 'o-')
plt.plot(h, mmag[h], 'o-')
plt.xlim([0, 0.05 * sr])
plt.legend(['mag', 'max_vals', 'hull'])
plt.subplot(122)
plt.plot(t, mmag)
plt.plot(t[max_times], max_vals, 'o-')
plt.plot(h / sr, mmag[h], 'o-')
#plt.xlim([1.9, 2.01])
print(h)

In [ ]:
# Describe note as initial amp + dB slope for single linear fit.
filt_len = 31
filt_win = np.hanning(filt_len)/np.sum(np.hanning(filt_len))

num_harmonics = mags.shape[0]
h_init_amp = np.zeros(num_harmonics)
h_decay = np.zeros(num_harmonics)
h_nums = range(num_harmonics)
for h in h_nums:
    mag_filt = lin_to_db(np.convolve(filt_win, db_to_lin(mags[h]), 'same'))
    mag_filt_trim = mag_filt[trim_ends_samps:-trim_ends_samps]
    t = np.arange(len(mag_filt_trim)) / sr
    linfit = lin_fit(mag_filt_trim)
    h_init_amp[h] = linfit[0]
    h_decay[h] = (linfit[-1] - linfit[0]) / (t[-1] - t[0])

plt.plot(h_nums, h_decay, '.', h_nums, h_init_amp, '.')
plt.legend(['decay dB/sec', 'init amp/dB'])
plt.grid()
_ = plt.title(filename + ' lin_fit harmonics')

In [ ]:
# Convert sinusoids into bp sets

def make_harms_params(mags, avg_freqs, max_harmonic=None, sr=44100, mag_floor=-100.0, terminal_slope=20.0):
  if not max_harmonic:
      max_harmonic = mags.shape[0]
    
  dt = 1 / sr
  sm_cyc = 2.0

  #trim_ends_sec = 0.014
  #trim_ends_samps = int(round(trim_ends_sec * sr))
  trim_start_samps = 128
  trim_end_samps = 1024
    
  harms_params = []

  for hnum in range(max_harmonic):
    mag_filt = lin_to_db(np.convolve(filt_win, db_to_lin(mags[hnum]), 'same'))
    mag_filt_trim = mag_filt[trim_start_samps : -trim_end_samps]
    #mag_filt_trim = mag_filt
    mmag = np.maximum(mag_floor, mag_filt_trim)
    anchor_times = hull(mmag, 12)
    anchor_vals = mmag[anchor_times]
    nvals = list(zip([0] + list(np.diff(anchor_times)), anchor_vals))
    bp_list = [(int(round(n * dt * 1000)), float(db_to_lin(val))) 
               for n, val in nvals]
    last_time, last_mag = bp_list[-1]
    last_mag_db = lin_to_db(last_mag)
    final_mag_db = mag_floor
    if last_mag_db > final_mag_db:
        terminal_slope = terminal_slope  # -db/sec
        final_dur = (last_mag_db - final_mag_db) / terminal_slope
        bp_list.append((int(round(1000 * final_dur)), float(db_to_lin(final_mag_db))))
    if bp_list[-1][1] == bp_list[-2][1]:  # Final points have same mag
        bp_list = bp_list[:-1]
    bp_list_pair = [float(avg_freqs[hnum]), bp_list]
    print("h", hnum + 1, bp_list_pair)
    harms_params.append(bp_list_pair)
  return harms_params

#harms_params = make_harms_params(mags_dict[filename], avg_freqs, mag_floor=-130)

In [ ]:
mags_dict = {}
harms_dict = {}

In [ ]:
# Filename to parameters set
#filenames = ['Piano.mf.C5.wav', 'Piano.ff.C5.wav', 'Piano.mf.D5.wav', 'Piano.ff.D5.wav']
filenames = ['Piano.mf.D4.wav', 'Piano.mf.D5.wav', 'Piano.ff.D4.wav', 'Piano.ff.D5.wav']

for filename in filenames:
    print(filename)
    d, sr = read_and_trim(filename, channel=0, rel_threshold=0.015, abs_threshold=0.001, duration=5.0)
    print(d.shape)
    mags_dict[filename], recons_d, freqs, avg_freqs = extract_harmonics(d, num_harms=12)
    #resid = d - recons_d
    harms_dict[filename] = make_harms_params(mags_dict[filename], avg_freqs, mag_floor=-130)

In [ ]:
# Mapping here is 1/8 shrunk after the first 250 ms.
def time_mapper(t, boundary=0.25, magnification=8.0):
        """Convert time in secs to a nonlinear projection."""
        #return np.log10(0.01 + t)
        t = np.asarray(t)
        magnified = (t < boundary)
        return magnified * t + (1 - magnified) * (boundary + (t - boundary) / magnification)

def plot_mags(mags, filename, nharms=None, hop_len=64, sr=44100, mag_floor=-130.0, mag_ceil=-20.0):
    if not nharms:
        nharms = mags.shape[0]
    nfrm = mags[:nharms, ::hop_len].shape[-1]
    frmTime = np.arange(nfrm) * hop_len / sr
    colors = plt.cm.rainbow(np.linspace(0, 1, max(12, nharms)))

    boundary = 0.25
    magnification = 8.0

    for i in range(nharms):
        plt.plot(time_mapper(frmTime[:nfrm]), mags[i, ::hop_len].T, '.', color=colors[i])
    plt.plot(time_mapper([boundary, boundary]), [mag_floor, mag_ceil], '--k')
    plt.ylim([mag_floor, mag_ceil])
    xtimes = np.hstack([np.arange(0, 0.25, 0.0625), np.arange(0.5, 5.0, 0.5)])
    plt.xlim([0, time_mapper(5.0)])
    plt.xticks(time_mapper(xtimes), xtimes)
    plt.legend(1 + np.arange(nharms), loc='upper right')
    _ = plt.title((filename + ' - ' + str(nharms) + ' harmonics'))
    

def plot_harms(harms_params, filename, nharms=None, hop_len=64, sr=44100, mag_floor=-130.0, mag_ceil=-20.0):
    if not nharms:
        nharms = len(harms_params)
    nfrm = mags[:nharms, ::hop_len].shape[-1]
    frmTime = np.arange(nfrm) * hop_len / sr
    colors = plt.cm.rainbow(np.linspace(0, 1, max(12, nharms)))

    # Mapping here is 1/8 shrunk after the first 250 ms.
    boundary = 0.25
    magnification = 8.0

    for i, hp in enumerate(harms_params):
        times, vals = zip(*np.array(hp[1]))
        times = np.cumsum(times) / 1000
        vals = np.array(vals)
        plt.fill_between(time_mapper(times), mag_floor, lin_to_db(vals), alpha=0.8, color=colors[i])
    plt.plot(time_mapper([boundary, boundary]), [mag_floor, -mag_ceil], '--k')
    plt.ylim([mag_floor, -mag_ceil])
    plt.legend(1 + np.arange(nharms))
    xtimes = np.hstack([np.arange(0, 0.25, 0.0625), np.arange(0.5, 5.0, 0.5)])
    plt.xticks(time_mapper(xtimes), xtimes)
    plt.xlim([0, time_mapper(5.0)])
    _ = plt.title((filename + ' - ' + str(nharms) + ' harmonics'))


In [ ]:
plt.figure(figsize=(15, 15))
for i, filename in enumerate(filenames):
    plt.subplot(2, 2, i + 1)
    plot_mags(mags_dict[filename], filename)
    #plot_harms(harms_dict[filename], filename)

In [ ]:
plt.figure(figsize=(15, 15))
for i, filename in enumerate(filenames):
    plt.subplot(2, 2, i + 1)
    #plot_mags(mags_dict[filename], filename)
    plot_harms(harms_dict[filename], filename)

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(projection='3d')

def time_mapper(t, boundary=0.25, magnification=8.0):
    """Convert time in secs to a nonlinear projection."""
    #return np.log10(0.01 + t)
    t = np.asarray(t)
    magnified = (t < boundary)
    return magnified * t + (1 - magnified) * (boundary + (t - boundary) / magnification)

from matplotlib.collections import PolyCollection

def polygon_under_graph(x, y):
    return [(x[0], 0.), *zip(x, y), (x[-1], 0.)]

verts = []
mag_floor = -130
for i, hp in list(enumerate(harms_dict[filename])):
        times, vals = zip(*np.array(hp[1]))
        times = np.cumsum(times) / 1000
        vals = np.array(vals)
        times = np.hstack([[times[0]], times])
        vals = np.hstack([[db_to_lin(mag_floor)], vals])
        #plt.plot(np.log10(0.01 + times), lin_to_db(vals), zs=-i, zdir='y', alpha=0.8, color=colors[i])
        #poly = ax.add_collection3d(plt.fill_between(2 + np.log10(0.01 + times), -100, lin_to_db(vals), alpha=0.8, color=colors[i]), 
        #                           zs=(i + 1), zdir='y')
        verts.append(polygon_under_graph(time_mapper(times), -mag_floor + lin_to_db(vals)))
facecolors = plt.colormaps['rainbow'](np.linspace(0, 1, len(verts)))
poly = PolyCollection(verts, facecolors=facecolors, alpha=.7)
ax.add_collection3d(poly, zs=np.arange(len(harms_dict[filename]), 0, -1), zdir='y')
ax.set_zlim(0, 120)

plt.title(filename)

#plt.zlim([-100, -20])
#plt.legend(1 + np.arange(nharms))
#plt.xticks(np.log10(0.01 + xtimes), xtimes)
#plt.xlim([0, 3000])
#_ = plt.title((filename + ' - ' + str(nharms) + ' harmonics'))
ax.view_init(elev=20., azim=-115, roll=0)

In [ ]:
harms_dict[filename][0]

In [ ]:
import json
print(filename.split('.'))
params_file = '.'.join(filename.split('.', -1)[:-1]) + '.json'
print(params_file)
with open(params_file, 'w') as f:
    f.write(json.dumps(harms_params))

In [ ]:
# Synthesize various versions.
# Original magnitudes, fixed frequencies

def synth_params(harms_params):
    result = np.zeros(0)
    for f0, hp in harms_params:
        # ignore first time, start list with first value.
        params = [lin_to_db(hp[0][1])]
        # Step through remaining pairs, converting times in ms to samples
        for bpp in hp[1:]:
            params.extend([int(round(bpp[0] * sr/1000)), lin_to_db(bpp[1])])
        # Expand (val, npoints, val, npoints..., val) list
        ep = expand_params(params)
        carrier = np.cos(2 * np.pi * f0 * np.arange(len(ep)) / sr)
        ep_len = len(ep)
        if ep_len > len(result):
            result = np.hstack([result, np.zeros(ep_len - len(result))])
        result[:ep_len] += db_to_lin(ep) * carrier
        print(f0)
    return result, ep

#audio, ep = synth_params([harms_params[i] for i in 1 + np.arange(10)])
filename = ['Piano.mf.C5.wav', 'Piano.ff.C5.wav', 'Piano.mf.D5.wav', 'Piano.ff.D5.wav'][3]
h_audio, ep = synth_params(harms_dict[filename][:12])

#f0 = harms_params[0][0]
#carrier = np.cos(2 * np.pi * f0 * np.arange(len(ep)) / sr)

#ii = np.arange(20000)
ii = np.arange(4000)
#plt.plot(t[ii], db_to_lin(mags[h_num - 1][ii]))
#plt.plot(t[ii], db_to_lin(ep[ii]))
#plt.title(filename)
npts = len(resid)
audio = h_audio[:npts] + resid
Audio(data=audio, rate=sr)
print(audio.shape, resid.shape)